<h1 align="center"> 
201902_Data Science Capstone_DATS_6501_10
</h1> 

<h4 align="center"> 
Author: Rajeev.Parvathaneni ([rajeev@gwu.edu](mailto:rajeev@gwu.edu))
</h4>


 I will try to make simple text summarizer that will summarize given Presedential speeches. The summarized reviews can be used as a reviews title also.I will use Spacy as natural language processing library for handling this project.
 In this program i  using basic NLP tools Spacy, NLTK & some python packages.( not using Nueral Networks , Tensorflow etc .. yet ) . 

# Object Of This Project 
The objective of this project is to build a model that can create relevant summaries from different President speeches.   

## What Is Text Summarization?
<img src="../images/TextSummarization.png">
Text summarization is the process of distilling the most important information from a source (or sources) to produce an abridged version for a particular user (or users) and task (or tasks).

## Types of Text Summarization Methods
Text summarization methods can be classified into different types.
<img src="../images/TextSummarization-flowchart.png">

**i. Based on input type:**

1. Single Document, where the input length is short. Many of the early summarization systems dealt with single document summarization.

2. Multi Document, where the input can be arbitrarily long.

**ii. Based on the purpose:**

1. Generic, where the model makes no assumptions about the domain or content of the text to be summarized and treats all inputs as homogeneous. The majority of the work that has been done revolves around generic summarization.

2. Domain-specific, where the model uses domain-specific knowledge to form a more accurate summary. For example, summarizing research papers of a specific domain, biomedical documents, etc.

3. Query-based, where the summary only contains information which answers natural language questions about the input text.

**iii. Based on output type:**

1. Extractive, where important sentences are selected from the input text to form a summary. Most summarization approaches today are extractive in nature.

2. Abstractive, where the model forms its own phrases and sentences to offer a more coherent summary, like what a human would generate. This approach is definitely a more appealing, but much more difficult than extractive summarization.

#  Import Packages 


In [2]:
import numpy as np # linear algebra
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display
import base64
import string
import re
from os import listdir
from os.path import isfile, join
from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from nltk.corpus import stopwords
import nltk
import heapq
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import csv
%matplotlib inline

stopwords = stopwords.words('english')
sns.set_context('notebook')

# Import Dataset 
> In this section, I will load the desired dataset for this notebook. This dataset has huge number of reviews. It will be hard to work with full dataset. So I will randomly sample the dataset into smaller chunks for easy purpose.

In [3]:
"""
These files have to move under src directory
"""
speeches = pd.read_csv("../data/Speeches.csv", 
                      nrows=5000,usecols =['title','content'],
                      na_values=['NA'] , encoding='iso-8859-1' )
speeches = speeches.dropna()
speeches_count = len(speeches.index)
speeches.head(15)

,title,content
0,"We are a strong nation, and we will maintain s...","For myself and for our Nation, I want to thank..."
1,From time to time we've been tempted to believ...,"Senator Hatfield, Mr. Chief Justice, Mr. Presi..."
2,"In another sense, our New Beginning is a conti...","Senator Mathias, Chief Justice Burger, Vice Pr..."
3,"A new breeze is blowing, and the old bipartisa...","Mr. Chief Justice, Mr. President, Vice Preside..."
4,"The world economy, the world environment, the ...","My fellow citizens, today we celebrate the mys..."
5,"With a new vision of Government, a new sense o...","My fellow citizens, at this last Presidential ..."
6,"To all nations, we will speak for the values t...","Thank you, all. Chief Justice Rehnquist, Presi..."
7,"From the viewpoint of centuries, the questions...","Vice President Cheney, Mr. Chief Justice, Pres..."
8,So to all the other peoples and governments wh...,"My fellow citizens, I stand here today humbled..."
9,For the American people can no more meet the d...,Thank you. Thank you so much.\nVice President ...


# Text preprocessing
> In this step, I will be using Spacy for preprocessing text, in others words I will clearing not useful features from reviews title like punctuation, stopwords. For this task, there are two useful libraries available in Python. 1. NLTK 2. Spacy. In this notebook, I will be working with Spacy because it is very fast and has many useful features compared to NLTK. So without further do let's get started!

In [4]:
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
def normalize_text(text):
    tm1 = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    tm2 = re.sub('<code>.*?</code>', '', tm1, flags=re.DOTALL)
    tm3 = re.sub('<[^>]+>©', '', tm1, flags=re.DOTALL)
    return tm3.replace("\n", "")

In [5]:
# in this step we are going to remove code syntax from text 
speeches['content_Cleaned_1'] = speeches['content'].apply(normalize_text)

In [6]:
print('Before normalizing text-----\n')
print(speeches['content'][2])
print('\nAfter normalizing text-----\n')
print(speeches['content_Cleaned_1'][2])

Before normalizing text-----

Senator Mathias, Chief Justice Burger, Vice President Bush, Speaker O'Neill, Senator Dole, reverend clergy, and members of my family and friends and my fellow citizens:
This day has been made brighter with the presence here of one who, for a time, has been absent. Senator John Stennis, God bless you and welcome back.
There is, however, one who is not with us today. Representative Gillis Long of Louisiana left us last night. And I wonder if we could all join in a moment of silent prayer.
[The President resumed speaking after a moment of silence. ]
Amen.
There are no words adequate to express my thanks for the great honor that you've bestowed on me. I'll do my utmost to be deserving of your trust.
This is, as Senator Mathias told us, the 50th time that we, the people, have celebrated this historic occasion. When the first President, George Washington, placed his hand upon the Bible, he stood less than a single day's journey by horseback from raw, untamed wil

God bless you, and God bless America.

After normalizing text-----



Senator Mathias, Chief Justice Burger, Vice President Bush, Speaker O'Neill, Senator Dole, reverend clergy, and members of my family and friends and my fellow citizens:This day has been made brighter with the presence here of one who, for a time, has been absent. Senator John Stennis, God bless you and welcome back.There is, however, one who is not with us today. Representative Gillis Long of Louisiana left us last night. And I wonder if we could all join in a moment of silent prayer.[The President resumed speaking after a moment of silence. ]Amen.There are no words adequate to express my thanks for the great honor that you've bestowed on me. I'll do my utmost to be deserving of your trust.This is, as Senator Mathias told us, the 50th time that we, the people, have celebrated this historic occasion. When the first President, George Washington, placed his hand upon the Bible, he stood less than a single day's journey by horseback from raw, untamed wilderness. There were 4 million Americ

We can see a huge difference after normalizing our text. Now we can see our text is more manageable. This will help us to explore the reviews and later making summarizer.

We are also seeing that there are some punctuation and stopwords. We also don't need them. In the first place, I don't remove them because we are gonna need this in future when we will make summarizer. So let's make another column that will store our normalized text without punctuation and stopwords.

## Clean text before feeding it to spaCy

In [7]:
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'
# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)
speeches['Content_Cleaned'] = speeches['content_Cleaned_1'].apply(lambda x: cleanup_text(x, False))

In [8]:
print('Speeches content with punctuatin and stopwords---\n')
print(speeches['content_Cleaned_1'][0])
print('\nSpeeches content after removing punctuation and stopwrods---\n')
print(speeches['Content_Cleaned'][0])

Speeches content with punctuatin and stopwords---

For myself and for our Nation, I want to thank my predecessor for all he has done to heal our land.In this outward and physical ceremony, we attest once again to the inner and spiritual strength of our Nation. As my high school teacher, Miss Julia Coleman, used to say, "We must adjust to changing times and still hold to unchanging principles."Here before me is the Bible used in the inauguration of our first President, in 1789, and I have just taken the oath of office on the Bible my mother gave me just a few years ago, opened to a timeless admonition from the ancient prophet Micah: "He hath showed thee, O man, what is good; and what doth the Lord require of thee, but to do justly, and to love mercy, and to walk humbly with thy God."This inauguration ceremony marks a new beginning, a new dedication within our Government, and a new spirit among us all. A President may sense and proclaim that new spirit, but only a people can provide it.T

nation want thank predecessor heal land . outward physical ceremony attest inner spiritual strength nation . high school teacher miss julia coleman use say must adjust change time still hold unchanging principles."here bible use inauguration first president 1789 take oath office bible mother give year ago open timeless admonition ancient prophet micah hath show thee man good doth lord require thee justly love mercy walk humbly thy god."this inauguration ceremony mark new beginning new dedication within government new spirit among . president may sense proclaim new spirit people provide . two century ago nation 's birth milestone long quest freedom . bold brilliant dream excite founder nation still await consummation . new dream set forth today rather urge fresh faith old dream . first society openly define term spirituality human liberty . unique self definition give exceptional appeal also impose special obligation take moral duty assume seem invariably good interest . give great resp

Wow! See! Now our text looks much readable and less messy!

# Content Summarizer
> In this step, I will try to make a content summarizer. There is a huge amount of research going for text summarization. But I will try to do a simple technique for text summarization. The technique describes below.

###  Convert Paragraphs to Sentences
> We first need to convert the whole paragraph into sentences. The most common way of converting paragraphs to sentences is to split the paragraph whenever a period is encountered.

###  Text Preprocessing
> After converting paragraph to sentences, we need to remove all the special characters, stop words and numbers from all the sentences.

###  Tokenizing the Sentences
> We need to tokenize all the sentences to get all the words that exist in the sentences

###  Find Weighted Frequency of Occurrence
> Next we need to find the weighted frequency of occurrences of all the words. We can find the weighted frequency of each word by dividing its frequency by the frequency of the most occurring word.

###  Replace Words by Weighted Frequency in Original Sentences
> The final step is to plug the weighted frequency in place of the corresponding words in original sentences and finding their sum. It is important to mention that weighted frequency for the words removed during preprocessing (stop words, punctuation, digits etc.) will be zero and therefore is not required to be added

###  Sort Sentences in Descending Order of Sum
> The final step is to sort the sentences in inverse order of their sum. The sentences with highest frequencies summarize the text.

In [9]:
# this is function for text summarization
def generate_summary(text_without_removing_dot, cleaned_text):
    sample_text = text_without_removing_dot
    doc = nlp(sample_text)
    sentence_list=[]
    for idx, sentence in enumerate(doc.sents): # we are using spacy for sentence tokenization
        sentence_list.append(re.sub(r'[^\w\s]','',str(sentence)))

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    for word in nltk.word_tokenize(cleaned_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)


    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]


    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    """
    Add president name  before each text 
    """
    print("Original Text:\n")
    print(text_without_removing_dot)
    print('\n\nSummarized text:\n')
    print(summary)  

Now we have written the function let's try to summarize some content.

In [10]:
"""
 do something here to show the data graphically or presentable 
"""
generate_summary(speeches['content_Cleaned_1'][10], speeches['Content_Cleaned'][10])

Original Text:



Chief Justice Roberts, President Carter, President Clinton, President Bush, President Obama, fellow Americans, and people of the world: Thank you.We, the citizens of America, are now joined in a great national effort to rebuild our country and restore its promise for all of our people. Together, we will determine the course of America and the world for many, many years to come. We will face challenges, we will confront hardships, but we will get the job done.Every 4 years, we gather on these steps to carry out the orderly and peaceful transfer of power, and we are grateful to President Obama and First Lady Michelle Obama for their gracious aid throughout this transition. They have been magnificent. Thank you.Today's ceremony, however, has very special meaning. Because today we are not merely transferring power from one administration to another or from one party to another, but we are transferring power from Washington, DC, and giving it back to you, the people.For too long, a small gr



Summarized text:

We the citizens of America are now joined in a great national effort to rebuild our country and restore its promise for all of our people We will get our people off of welfare and back to work rebuilding our country with American hands and American labor Chief Justice Roberts President Carter President Clinton President Bush President Obama fellow Americans and people of the world Thank you We assembled here today are issuing a new decree to be heard in every city in every foreign capital and in every hall of power When America is united America is totally unstoppable Together we will determine the course of America and the world for many many years to come We share one heart one home and one glorious destiny


   That's awesome first step for Summarized Text !!